# Importing Required Modules

In [247]:
import cv2
import numpy as np
import requests
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA


# Downloading the HAAR Data File

In [248]:
def gethaar(url="https://raw.githubusercontent.com/opencv/opencv/22f0bcaf8f4fb406ae251830cf99b7ea05de40e4/data/haarcascades/haarcascade_frontalface_default.xml"):    
    haar = requests.get(url)

# Saving the HAAR Data File

In [249]:
def savehaar():
    datafile = open("data.xml", "wb")
    datafile.write(haar.content)
    datafile.close()

# Using the Classifier to get the Face Cooridinate

In [250]:
def classify():
    haar_data = cv2.CascadeClassifier('data.xml')

# Using the Default Video Capture Device

In [251]:
def setcam():
    capture = cv2.VideoCapture(0)
    return capture
def releasecam(capture):
    capture.release()

# Storing the Mask And Umasked Data

In [252]:
def facedata(data):
    capture = setcam()
    while True:
        flag, img = capture.read()
        if flag:
            faces = haar_data.detectMultiScale(img)
            for x, y, w, h in faces:
                cv2.rectangle(img, (x,y), (x+w, y+h), (255,0,255), 4)
                face = img[y:y+h, x:x+w, :]
                face = cv2.resize(face, (50, 50))
                if len(data)<250:
                    data.append(face)
            cv2.imshow("Seal's Face Mask Detector", img)
            if cv2.waitKey(10) == 27 or len(data)>=250:
                break
    releasecam(capture)
    cv2.destroyAllWindows()
    return data


# Storing Unmasked Data

In [253]:
def unmasked():
    print("Don't Wear A Mask. Press Escape to Exit.")
    data = facedata([])
    np.save('without_mask.npy', data)
    return True

# Storing Masked Data

In [254]:
def masked():
    print("Wear A Mask. Press Escape to Exit.")
    data = facedata([])
    np.save('with_mask.npy', data)
    return True

# Loading the saved files

In [255]:
def load():
    with_mask = np.load("with_mask.npy")
    without_mask = np.load("without_mask.npy")


# Reshaping the saved files

In [256]:
def reshape():
    with_mask = with_mask.reshape(250, 50*50*3)
    without_mask = without_mask.reshape(250, 50*50*3)

# Combining the Reshaped Files

In [257]:
def combine(lst1, lst2):
    X = np.r_[lst1, lst2]
    return X

# Segregating the Dependent Variable 

In [258]:
def divide():
    labels = np.zeros(X.shape[0])
    labels[250:] = 1.0
    names = {0: 'Mask', 1: 'No Mask'}
    return labels

# Fitting Data

In [259]:
def fit(n=3):
    pca = PCA(n_components=n)
    X = pca.fit_transform(X)
    return X

# Training

In [260]:
def train(x_train, y_train):
    svm = SVC()
    svm.fit(x_train, y_train)

# Predicting

In [261]:
def predict(var):
    y_pred = svm.predict(var)
    return y_pred

# Calculating Accuracy

In [262]:
def accuracy(y_train, y_pred):
    return accuracy_score(y_train, y_pred)


# Driver Function

In [263]:
def resetconfirmation():
    return "n"
def main():
    url = "https://raw.githubusercontent.com/opencv/opencv/22f0bcaf8f4fb406ae251830cf99b7ea05de40e4/data/haarcascades/haarcascade_frontalface_default.xml"
    gethaar(url)
    savehaar()
    classify()
    confirm = resetconfirmation()
    while confirm.lower() != "y":
        print("Remove a Mask if you are wearing one.\nSit In a well-lit environment.\nPress Y to confirm: ")
        confirm = input()
    unmasked()
    confirm = resetconfirmation()
    while confirm.lower() != "y":
        print("Wear a Mask if you are not wearing one.\nSit In a well-lit environment.\nPress Y to confirm: ")
        confirm = input()
    masked()
    load()
    reshape()
    X = combine(with_mask, without_mask)
    labels = divide()
    X = fit()
    train(X, lables)
    y_pred = predict(X)
    confidence = accuracy(labels, y_pred)
    print(ypred, confidence)


# Calling Main

In [264]:
if __name__ == "__main__":
    pass
    main()

Remove a Mask if you are wearing one.
Sit In a well-lit environment.
Press Y to confirm: 
Remove a Mask if you are wearing one.
Sit In a well-lit environment.
Press Y to confirm: 
Don't Wear A Mask. Press Escape to Exit.
Wear a Mask if you are not wearing one.
Sit In a well-lit environment.
Press Y to confirm: 
Wear A Mask. Press Escape to Exit.


UnboundLocalError: local variable 'with_mask' referenced before assignment